In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import json

In [2]:
movies = pd.read_csv('./ml-20m/movies.csv')
ratings = pd.read_csv('./ml-20m/ratings.csv')
# tags = pd.read_csv('./ml-20m/tags.csv')
# links = pd.read_csv('./ml-20m/links.csv')
# genome_scores = pd.read_csv('./ml-20m/genome-scores.csv')
# genome_tags = pd.read_csv('./ml-20m/genome-tags.csv')

In [22]:
# Calculate the average rating and number of ratings for each movie
merge_rate_movie = pd.merge(ratings,movies,how='left',on='movieId')
merge_rate_movie

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
20000258,138493,68954,4.5,1258126920,Up (2009),Adventure|Animation|Children|Drama
20000259,138493,69526,4.5,1259865108,Transformers: Revenge of the Fallen (2009),Action|Adventure|Sci-Fi|IMAX
20000260,138493,69644,3.0,1260209457,Ice Age: Dawn of the Dinosaurs (2009),Action|Adventure|Animation|Children|Comedy|Rom...
20000261,138493,70286,5.0,1258126944,District 9 (2009),Mystery|Sci-Fi|Thriller


In [24]:
number_ratings = merge_rate_movie.groupby(['movieId']).count()['userId']
number_ratings

movieId
1         49695
2         22243
3         12735
4          2756
5         12161
          ...  
131254        1
131256        1
131258        1
131260        1
131262        1
Name: userId, Length: 26744, dtype: int64

In [25]:
average_ratings = merge_rate_movie.groupby('movieId')['rating'].mean()
average_ratings

movieId
1         3.921240
2         3.211977
3         3.151040
4         2.861393
5         3.064592
            ...   
131254    4.000000
131256    4.000000
131258    2.500000
131260    3.000000
131262    4.000000
Name: rating, Length: 26744, dtype: float64

In [26]:
merge_rate_movie_ratings = pd.merge(average_ratings,number_ratings,how='left',on="movieId")
merge_rate_movie_ratings.columns = ['avgRating', 'moiveCount']
merge_rate_movie_ratings

,avgRating,moiveCount
movieId,,
1,3.921240,49695
2,3.211977,22243
3,3.151040,12735
4,2.861393,2756
5,3.064592,12161
...,...,...
131254,4.000000,1
131256,4.000000,1
131258,2.500000,1


In [27]:
merge_rate_movie = pd.merge(merge_rate_movie,merge_rate_movie_ratings,how='left',on="movieId")
merge_rate_movie

,userId,movieId,rating,timestamp,title,genres,avgRating,moiveCount
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy,3.211977,22243
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,3.952230,8520
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,3.898055,44980
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.053493,43249
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.334372,47006
...,...,...,...,...,...,...,...,...
20000258,138493,68954,4.5,1258126920,Up (2009),Adventure|Animation|Children|Drama,4.038266,9264
20000259,138493,69526,4.5,1259865108,Transformers: Revenge of the Fallen (2009),Action|Adventure|Sci-Fi|IMAX,2.938121,2012
20000260,138493,69644,3.0,1260209457,Ice Age: Dawn of the Dinosaurs (2009),Action|Adventure|Animation|Children|Comedy|Rom...,3.387130,979
20000261,138493,70286,5.0,1258126944,District 9 (2009),Mystery|Sci-Fi|Thriller,3.872481,8038


In [42]:
# get all count more than 50's movies
mostRatedMovies = merge_rate_movie[merge_rate_movie['moiveCount'] > 500][:]

In [43]:
mostRatedMovies.dtypes

userId          int64
movieId         int64
rating        float64
timestamp       int64
title          object
genres         object
avgRating     float64
moiveCount      int64
dtype: object

In [44]:
mostRatedMovies

,userId,movieId,rating,timestamp,title,genres,avgRating,moiveCount
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy,3.211977,22243
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,3.952230,8520
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,3.898055,44980
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.053493,43249
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.334372,47006
...,...,...,...,...,...,...,...,...
20000257,138493,68319,4.5,1260209720,X-Men Origins: Wolverine (2009),Action|Sci-Fi|Thriller,3.325948,2821
20000258,138493,68954,4.5,1258126920,Up (2009),Adventure|Animation|Children|Drama,4.038266,9264
20000259,138493,69526,4.5,1259865108,Transformers: Revenge of the Fallen (2009),Action|Adventure|Sci-Fi|IMAX,2.938121,2012
20000260,138493,69644,3.0,1260209457,Ice Age: Dawn of the Dinosaurs (2009),Action|Adventure|Animation|Children|Comedy|Rom...,3.387130,979


In [47]:
len(mostRatedMovies['title'].unique())

4483

In [48]:
len(mostRatedMovies['userId'].unique())

138493

In [49]:
moviePivot = mostRatedMovies.pivot_table(
    index=["title"],
    columns=["userId"],
    values=["rating"],
    fill_value=0,
)

In [65]:
moviePivot

rating                                     \
userId                         1      2      3      4      5      6        
title                                                                      
'Til There Was You (1997)         0.0      0      0      0      0      0   
'burbs, The (1989)                0.0      0      0      0      0      0   
(500) Days of Summer (2009)       0.0      0      0      0      0      0   
*batteries not included (1987)    0.0      0      0      0      0      0   
...And Justice for All (1979)     0.0      0      0      0      0      0   
...                               ...    ...    ...    ...    ...    ...   
[REC] (2007)                      0.0      0      0      0      0      0   
eXistenZ (1999)                   0.0      0      0      0      0      0   
xXx (2002)                        0.0      0      0      0      0      0   
xXx: State of the Union (2005)    0.0      0      0      0      0      0   
¡Three Amigos! (1986)             0.0      0      0      0      0      0   

                                                            ...                \
userId                         7      8      9      10      ... 138484 138485   
title                                                       ...                 
'Til There Was You (1997)           0      0      0      0  ...      0    0.0   
'burbs, The (1989)                  0      0      0      0  ...      0    0.0   
(500) Days of Summer (2009)         0      0      0      0  ...      0    0.0   
*batteries not included (1987)      0      0      0      0  ...      0    0.0   
...And Justice for All (1979)       0      0      0      0  ...      0    0.0   
...                               ...    ...    ...    ...  ...    ...    ...   
[REC] (2007)                        0      0      0      0  ...      0    0.0   
eXistenZ (1999)                     0      0      0      0  ...      0    0.0   
xXx (2002)                          0      0      0      0  ...      0    0.0   
xXx: State of the Union (2005)      0      0      0      0  ...      0    0.0   
¡Three Amigos! (1986)               2      0      0      0  ...      0    0.0   

                                                                          \
userId                         138486 138487 138488 138489 138490 138491   
title                                                                      
'Til There Was You (1997)         0.0      0      0    0.0      0    0.0   
'burbs, The (1989)                0.0      0      0    0.0      0    0.0   
(500) Days of Summer (2009)       0.0      0      0    0.0      0    0.0   
*batteries not included (1987)    0.0      0      0    0.0      0    0.0   
...And Justice for All (1979)     0.0      0      0    0.0      0    0.0   
...                               ...    ...    ...    ...    ...    ...   
[REC] (2007)                      0.0      0      0    0.0      0    0.0   
eXistenZ (1999)                   0.0      0      0    0.0      0    0.0   
xXx (2002)                        0.0      0      0    0.0      0    0.0   
xXx: State of the Union (2005)    0.0      0      0    0.0      0    0.0   
¡Three Amigos! (1986)             0.0      0      0    0.0      0    0.0   

                                              
userId                         138492 138493  
title                                         
'Til There Was You (1997)         0.0    0.0  
'burbs, The (1989)                0.0    0.0  
(500) Days of Summer (2009)       0.0    0.0  
*batteries not included (1987)    0.0    0.0  
...And Justice for All (1979)     0.0    0.0  
...                               ...    ...  
[REC] (2007)                      0.0    0.0  
eXistenZ (1999)                   0.0    0.0  
xXx (2002)                        0.0    0.0  
xXx: State of the Union (2005)    0.0    0.0  
¡Three Amigos! (1986)             0.0    0.0  

[4483 rows x 138493 columns]

In [66]:
import pandas as pd
import os
# 创建hdf文件
h5_store = pd.HDFStore('moviePivot.h5', mode='w')
h5_store['moviePivot'] = moviePivot
# 关闭hdf文件
h5_store.close()

In [55]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

movie_features_df_matrix = csr_matrix(moviePivot.values)

movie_features_df_matrix

<4483x138493 sparse matrix of type '<class 'numpy.float64'>'
	with 18714467 stored elements in Compressed Sparse Row format>

In [56]:
"""使用整个数据集拟合最终的无监督模型，以找到每一个最相似的电影"""
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=11, n_jobs=-1)
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=11, p=2,
                 radius=1.0)

In [60]:
# 选择一个标题
favoriteMovie = 'Iron Man (2008)'
query_index = moviePivot.index.get_loc(favoriteMovie)
distances, indices = model_knn.kneighbors(moviePivot.loc[favoriteMovie,:].values.reshape(1, -1), n_neighbors=11)
 
# 根据kNN模型打印10部最相似的电影
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(moviePivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, moviePivot.index[indices.flatten()[i]], distances.flatten()[i]))




Recommendations for Iron Man (2008):

1: Dark Knight, The (2008), with distance of 0.3742058573336221:
2: Star Trek (2009), with distance of 0.4132744620437958:
3: WALL·E (2008), with distance of 0.4288254076921597:
4: Batman Begins (2005), with distance of 0.44117734734015845:
5: Bourne Ultimatum, The (2007), with distance of 0.4508639925223026:
6: Casino Royale (2006), with distance of 0.4576121616203074:
7: Avatar (2009), with distance of 0.4614256282983711:
8: Iron Man 2 (2010), with distance of 0.46337905754914765:
9: V for Vendetta (2006), with distance of 0.48080602167269393:
10: District 9 (2009), with distance of 0.48661203080569715:


In [61]:
indices

array([[2087, 1017, 3790, 4263,  402,  609,  746,  325, 2088, 4218, 1128]])

In [62]:
moviePivot.to_csv('moviePivot.csv', index =False)

In [63]:
from sklearn.externals import joblib
joblib.dump(model_knn, "model_knn.m")

/Users/ly/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['model_knn.m']

In [69]:
m = joblib.load("model_knn.m")
m

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=11, p=2,
                 radius=1.0)

In [74]:
h5_store = pd.HDFStore('moviePivot.h5', mode='r')
h5_store['moviePivot']

rating                                     \
userId                         1      2      3      4      5      6        
title                                                                      
'Til There Was You (1997)         0.0      0      0      0      0      0   
'burbs, The (1989)                0.0      0      0      0      0      0   
(500) Days of Summer (2009)       0.0      0      0      0      0      0   
*batteries not included (1987)    0.0      0      0      0      0      0   
...And Justice for All (1979)     0.0      0      0      0      0      0   
...                               ...    ...    ...    ...    ...    ...   
[REC] (2007)                      0.0      0      0      0      0      0   
eXistenZ (1999)                   0.0      0      0      0      0      0   
xXx (2002)                        0.0      0      0      0      0      0   
xXx: State of the Union (2005)    0.0      0      0      0      0      0   
¡Three Amigos! (1986)             0.0      0      0      0      0      0   

                                                            ...                \
userId                         7      8      9      10      ... 138484 138485   
title                                                       ...                 
'Til There Was You (1997)           0      0      0      0  ...      0    0.0   
'burbs, The (1989)                  0      0      0      0  ...      0    0.0   
(500) Days of Summer (2009)         0      0      0      0  ...      0    0.0   
*batteries not included (1987)      0      0      0      0  ...      0    0.0   
...And Justice for All (1979)       0      0      0      0  ...      0    0.0   
...                               ...    ...    ...    ...  ...    ...    ...   
[REC] (2007)                        0      0      0      0  ...      0    0.0   
eXistenZ (1999)                     0      0      0      0  ...      0    0.0   
xXx (2002)                          0      0      0      0  ...      0    0.0   
xXx: State of the Union (2005)      0      0      0      0  ...      0    0.0   
¡Three Amigos! (1986)               2      0      0      0  ...      0    0.0   

                                                                          \
userId                         138486 138487 138488 138489 138490 138491   
title                                                                      
'Til There Was You (1997)         0.0      0      0    0.0      0    0.0   
'burbs, The (1989)                0.0      0      0    0.0      0    0.0   
(500) Days of Summer (2009)       0.0      0      0    0.0      0    0.0   
*batteries not included (1987)    0.0      0      0    0.0      0    0.0   
...And Justice for All (1979)     0.0      0      0    0.0      0    0.0   
...                               ...    ...    ...    ...    ...    ...   
[REC] (2007)                      0.0      0      0    0.0      0    0.0   
eXistenZ (1999)                   0.0      0      0    0.0      0    0.0   
xXx (2002)                        0.0      0      0    0.0      0    0.0   
xXx: State of the Union (2005)    0.0      0      0    0.0      0    0.0   
¡Three Amigos! (1986)             0.0      0      0    0.0      0    0.0   

                                              
userId                         138492 138493  
title                                         
'Til There Was You (1997)         0.0    0.0  
'burbs, The (1989)                0.0    0.0  
(500) Days of Summer (2009)       0.0    0.0  
*batteries not included (1987)    0.0    0.0  
...And Justice for All (1979)     0.0    0.0  
...                               ...    ...  
[REC] (2007)                      0.0    0.0  
eXistenZ (1999)                   0.0    0.0  
xXx (2002)                        0.0    0.0  
xXx: State of the Union (2005)    0.0    0.0  
¡Three Amigos! (1986)             0.0    0.0  

[4483 rows x 138493 columns]

In [75]:
h5_store.close()

In [76]:
from sklearn.externals import joblib
m = joblib.load("model_knn.m")
h5_store = pd.HDFStore('moviePivot.h5', mode='r')
p = h5_store['moviePivot']
h5_store.close()
print('Already Read')
# 选择一个标题
favoriteMovie = 'Iron Man (2008)'
query_index = p.index.get_loc(favoriteMovie)
distances, indices = m.kneighbors(p.loc[favoriteMovie,:].values.reshape(1, -1), n_neighbors=11)
 
# 根据kNN模型打印10部最相似的电影
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(p.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, p.index[indices.flatten()[i]], distances.flatten()[i]))


Already Read
Recommendations for Iron Man (2008):

1: Dark Knight, The (2008), with distance of 0.3742058573336221:
2: Star Trek (2009), with distance of 0.4132744620437958:
3: WALL·E (2008), with distance of 0.4288254076921597:
4: Batman Begins (2005), with distance of 0.44117734734015845:
5: Bourne Ultimatum, The (2007), with distance of 0.4508639925223026:
6: Casino Royale (2006), with distance of 0.4576121616203074:
7: Avatar (2009), with distance of 0.4614256282983711:
8: Iron Man 2 (2010), with distance of 0.46337905754914765:
9: V for Vendetta (2006), with distance of 0.48080602167269393:
10: District 9 (2009), with distance of 0.48661203080569715:


In [77]:
m

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=11, p=2,
                 radius=1.0)

In [2]:
import pandas as pd
h5_store = pd.HDFStore('moviePivot.h5', mode='r')
p = h5_store['moviePivot']
h5_store.close()

["'Til There Was You (1997)",
 "'burbs, The (1989)",
 '(500) Days of Summer (2009)',
 '*batteries not included (1987)',
 '...And Justice for All (1979)',
 '10 (1979)',
 '10 Things I Hate About You (1999)',
 '10,000 BC (2008)',
 '101 Dalmatians (1996)',
 '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
 '102 Dalmatians (2000)',
 '11:14 (2003)',
 '12 Angry Men (1957)',
 '12 Years a Slave (2013)',
 '127 Hours (2010)',
 '13 Ghosts (1960)',
 '13 Going on 30 (2004)',
 '13th Warrior, The (1999)',
 '1408 (2007)',
 '15 Minutes (2001)',
 '16 Blocks (2006)',
 '17 Again (2009)',
 '187 (One Eight Seven) (1997)',
 '1941 (1979)',
 '1984 (Nineteen Eighty-Four) (1984)',
 '2 Days in Paris (2007)',
 '2 Days in the Valley (1996)',
 '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)',
 '20 Dates (1998)',
 '20,000 Leagues Under the Sea (1954)',
 '200 Cigarettes (1999)',
 '2001: A Space Odyssey (1968)',
 '2010: The Year We Make Contact (1984)',
 '2012 (2009)',
 '2046 (2004)',
 '21 (2008)',
 '21 

In [16]:
quantile_df = mostRatedMovies.quantile(q = [0.25, 0.5, 0.75], axis='index', numeric_only=True, interpolation='linear')
quantile_df

,userId,movieId,rating,timestamp,avgRating,moiveCount
0.25,34443.0,296.0,3.5,8.800149e+08,4.029000,52244.0
0.50,69265.0,356.0,4.0,1.044565e+09,4.174231,59715.0
0.75,103833.0,589.0,5.0,1.206286e+09,4.190672,63366.0


In [17]:
mean_avgRating = mostRatedMovies['avgRating'].mean()
mean_avgRating

4.115882701776845

In [18]:
quantile_moiveCount = list(quantile_df['moiveCount'])[2]
quantile_moiveCount

63366.0

In [19]:
mostRatedMovies['weightedAvgRating'] = ((mostRatedMovies['avgRating']*mostRatedMovies['moiveCount']) + (mean_avgRating*quantile_moiveCount)) / (mostRatedMovies['moiveCount']+quantile_moiveCount)
mostRatedMovies

,userId,movieId,rating,timestamp,title,genres,avgRating,moiveCount,weightedAvgRating
9,1,260,4.0,1112484826,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.190672,54502,4.150465
11,1,296,4.0,1112484767,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.174231,67310,4.145937
12,1,318,4.0,1112484798,"Shawshank Redemption, The (1994)",Crime|Drama,4.446990,63366,4.281437
16,1,589,3.5,1112485557,Terminator 2: Judgment Day (1991),Action|Sci-Fi,3.931954,52244,4.032766
17,1,593,3.5,1112484661,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.177057,63299,4.146453
...,...,...,...,...,...,...,...,...,...
19999907,138493,296,4.5,1255810263,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.174231,67310,4.145937
19999912,138493,356,5.0,1255817103,Forrest Gump (1994),Comedy|Drama|Romance|War,4.029000,66172,4.071500
19999919,138493,480,4.5,1255817600,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.664741,59715,3.897003
19999923,138493,527,5.0,1255817541,Schindler's List (1993),Drama|War,4.310175,50054,4.201627


In [21]:
mostRatedMovies.to_csv('movie_rate_with_weight.csv', index =False)